In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

import numpy as np
import pandas as pd

import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torch.utils.data import DataLoader
from utils import MultipleRegressionWithSoftmax, EmbeddingsDataset2

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import r2_score, accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
from scipy.stats import pearsonr

In [ ]:
n_classes = 2
modality = 'music' # 'music', 'speech', or 'video'
which = 'openl3' # 'mfcc', 'msd' or 'openl3' for music, 'slow_fast' for video, 'hubert' for speech
voice = True 

fn_suffix = {
    'music': {
        'mfcc': '',
        'msd': '_backend', 
        'openl3': '_music', # '_music' or '_env'
    },
    'video': {
        'slow_fast': '_slow', # '_slow' or '_fast'
    },
    'speech': {
        'hubert': '_wave_encoder', # '_wave_encoder' or '_transformer'
    }
}

embedding_dimensions = {
    'video': {
        'slow_fast': 2048 if fn_suffix['video']['slow_fast']=='_slow' else 256,
    },
    'music': {
        'mfcc': 60,
        'msd': 256,
        'openl3': 512,
    },
    'speech': {
        'hubert': 1024 if fn_suffix['speech']['hubert']=='_transformer' else 512,
    }
}

## Load ground truth

In [ ]:
groundtruth_df = pd.read_csv("groundtruth_merged.csv")
groundtruth_df.set_index("stimulus_id", inplace=True)
groundtruth_df.head()

In [ ]:
mid_level_features = pd.read_csv("mid_level_features.csv").drop(columns=["target"])
mid_level_features.set_index("stimulus_id", inplace=True)

# drop columns that are not significant based on the ANOVA test in the music cognition paper
todrop = ["Wide pitch variation/Narrow pitch variation", "Repetitive/Non-repetitive", "Fast tempo/Slow tempo"]
mid_level_features = mid_level_features.drop(columns=todrop)

# perform min-max scaling [0, 1] of all mid-level features
mid_level_features = (mid_level_features - mid_level_features.min()) / (mid_level_features.max() - mid_level_features.min())
mid_level_features.head()

In [ ]:
not_found = 0
for stimulus_id in groundtruth_df.index:
    if not os.path.exists(f"{modality}/embeddings_{which}/{stimulus_id}{fn_suffix[modality][which]}.npy"):
        print(f"Embedding for {stimulus_id} not found")
        not_found += 1

assert not_found == 0

## Load embeddings

In [ ]:
embedding_dim = embedding_dimensions[modality][which]

X = np.empty((groundtruth_df.shape[0], embedding_dim))
y_reg = np.empty((mid_level_features.shape[0], mid_level_features.shape[1]))

for i,stimulus_id in enumerate(groundtruth_df.index):
    embedding = np.load(f"{modality}/embeddings_{which}{'' if voice else '_novoice'}/" +
                        f"{stimulus_id}{fn_suffix[modality][which]}.npy")
    X[i] = embedding.mean(axis=0)
    y_reg[i] = mid_level_features.loc[stimulus_id].values

X.shape, y_reg.shape

In [ ]:
classes = ["Girls/women", "Boys/men"] if n_classes==2 else ["Girls/women", "Mixed", "Boys/men"]
# mask = groundtruth_df.target.isin(classes) 

y_cls = groundtruth_df.target.values

# convert to integers, and when the classes are not in 'classes' set them to -1
y_cls = groundtruth_df.target.values
y_cls = [classes.index(x) if x in classes else -1 for x in y_cls]
y_cls = np.array(y_cls)

pd.Series(y_cls).value_counts()

### partially annotated data will be handeled by the model, see utils.py

In [ ]:
params = {
    "input_dim": X.shape[1], 
    "n_regressions": y_reg.shape[1], 
    "output_dim": n_classes
    }

kf = KFold(n_splits=5, shuffle=True, random_state=42)

accuracies = []
f1s = []
r2s = []
pearsons = []
for train_index, test_index in kf.split(X):

    test_index, val_index = train_test_split(test_index, test_size=0.5, random_state=42)
    
    X_train, X_test, X_val = X[train_index], X[test_index], X[val_index]
    y_reg_train, y_reg_test, y_reg_val = y_reg[train_index], y_reg[test_index], y_reg[val_index]
    y_cls_train, y_cls_test, y_cls_val = y_cls[train_index], y_cls[test_index], y_cls[val_index]

    train_dataset = EmbeddingsDataset2(X_train, y_reg_train, y_cls_train)
    val_dataset = EmbeddingsDataset2(X_val, y_reg_val, y_cls_val)
    test_dataset = EmbeddingsDataset2(X_test, y_reg_test, y_cls_test)
    
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=1)
    val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=1)
    test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=1)

    model = MultipleRegressionWithSoftmax(**params)
    
    checkpoint_callback = ModelCheckpoint(monitor='val_loss')
    trainer = pl.Trainer(max_epochs=200,
                        callbacks=[checkpoint_callback, EarlyStopping(monitor='val_loss', patience=50)],
                        enable_progress_bar = False,
                        accelerator='gpu',
                        devices=1)
    trainer.fit(model, train_loader, test_loader)

    # load best model
    model = model.load_from_checkpoint(checkpoint_callback.best_model_path, **params)
    
    # evaluate
    skip_unlabelled = y_cls_test != -1

    model.eval()
    with torch.no_grad():
        y_reg_pred, out_cls = model(torch.from_numpy(X_test).float())
    
    y_cls_pred = torch.argmax(out_cls, dim=1).numpy()[skip_unlabelled]
    
    accuracies.append(accuracy_score(y_cls_test[skip_unlabelled], y_cls_pred))
    f1s.append(f1_score(y_cls_test[skip_unlabelled], y_cls_pred, average='weighted'))

    r2_values = r2_score(y_reg_test, y_reg_pred, multioutput='raw_values')
    r2s.append(r2_values)

    r = [pearsonr(y_reg_test[:,i], y_reg_pred[:,i])[0] for i in range(y_reg_test.shape[1])]
    pearsons.append(r)


In [ ]:
print(f"Accuracy: {np.mean(accuracies):.2f} ± {np.std(accuracies):.2f}")
print(f"F1: {np.mean(f1s):.2f} ± {np.std(f1s):.2f}")

In [ ]:
# transpose to get r2s per feature
r2s = np.array(r2s).T

for i, r2 in enumerate(r2s):
    print(f"R2 for {mid_level_features.columns[i]}: {np.mean(r2):.2f} ± {np.std(r2):.2f}")

In [ ]:
# transpose to get pearsons per feature
pearsons = np.array(pearsons).T
#
for i, r in enumerate(pearsons):
    print(f"Pearson's r for {mid_level_features.columns[i]}: {np.mean(r):.2f} ± {np.std(r):.2f}")